# Export community graph and GSEA bar plot for largest community
### Aim:
Export the graph community in .gml format for plotting in Gephi
Perform GSEA for largest community and plot bar plot

### Output:
Community graph in .gml file
GSEA bar plot for largest community

In [15]:
# Import packages and DepMap tools
from DepMapTools.DataImport import SaveLoad, DataDownload
from DepMapTools.GeneOntology import OntologyAnalysis
import networkx as nx
import os
import gseapy as gp

In [16]:
# Define CSV file path and load clean Achilles CRISPR data
dd = DataDownload()
PRD = ".."
csv_path = os.path.join(PRD,
                        'AnalysisData')
df = dd.load_data('CRISPR_gene_effect_clean.csv', 0, csv_path)

In [17]:
# Instantiate SaveLoad class and define directory to save network image
sl = SaveLoad()
oa = OntologyAnalysis()
dir_name = 'Figures'
data_path = os.path.join(PRD, dir_name)
os.makedirs(data_path,
            exist_ok=True)

In [18]:
# Load single gene analysis dictionary
filename = 'chronos_singlegene_funk.pickle'
pickle_path = os.path.join(PRD,
                           '1_SingleGene_Analysis/pickle_files/' + filename)
funk_net = sl.load_dict_pickle(pickle_path)

In [19]:
# Load com analysis dictionary
filename = 'chronos_k_community_funk.pickle'
pickle_path = os.path.join(PRD,
                           '2_Community_Analysis/pickle_files/' + filename)
coms = sl.load_dict_pickle(pickle_path)

In [20]:
# Input gene to plot and export community graph
gene = 'INHBE'

In [13]:
# Define function to plot GSEA and export the community graph
def plot_export_community(gene, net_dict, com_dict):
    g = net_dict[gene]['Network']
    com = com_dict[gene]['Community']
    com_genes = com['gene'].tolist()
    com_genes.append(gene)
    g_nodes = list(g.nodes())
    to_del = []
    # Remove nodes not in the community
    for n in g_nodes:
        if n not in com_genes:
            to_del.append(n)
    for x in to_del:
        g.remove_node(x)
    # Export community graph
    nx.write_gml(g, f'{gene}_com.gml')
    # Perform GSEA for and plot graph
    com_gsea = oa.gsea(com_genes)
    gp.barplot(com_gsea,
               title=f'{gene} Community GSEA',
               top_term=10,
               color='#27408B',
               figsize=(8, 8),
               ofname=f'{gene}_bar')

In [14]:
plot_export_community(gene, funk_net, coms)